# Query data from overpassAPI (openstreetmaps)


query now with Python by using the requests package in order to access the Overpass AP

## Area[] Berlin 
area["ISO3166-2"="DE-BE"]
https://www.openstreetmap.org/relation/62422

## admin_level=4
admin_level=2 through admin_level=10 allow for different administrative subdivision schemes to be handled in a consistent way by data consumers. The use of a numbering scheme rather than words for the values helps avoid confusion due to different terminology used in different countries
A data consumer looking for municipalities corresponding to "city", "town" or "village" boundaries will find these tagged anywhere from admin_level=4 (e.g. relation Berlin)

In [ ]:
import requests
import json

overpass_url = "http://overpass-api.de/api/interpreter"
overpass_query = """
[out:json];
area["ISO3166-2"="DE-BE"][admin_level=4];
(node['shop'='convenience'](area);
 way['shop'='convenience' ](area);
 rel['shop'='convenience' ](area);
);
out center;
"""
response = requests.get(overpass_url, 
                        params={'data': overpass_query})
data = response.json()

In [ ]:
overpass_query

In [ ]:
data

## Plotting 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Collect coords into list
coords = []
for element in data['elements']:
  if element['type'] == 'node':
    lon = element['lon']
    lat = element['lat']
    coords.append((lon, lat))
  elif 'center' in element:
    lon = element['center']['lon']
    lat = element['center']['lat']
    coords.append((lon, lat))# Convert coordinates into numpy array
X = np.array(coords)

plt.plot(X[:, 0], X[:, 1], 'o')
plt.title('Spatkauf in Berlin')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.axis('equal')
plt.show()

In [ ]:
kkk

## params function

In [ ]:
#not working
# def param_key(keys):
    
#     #params defining the whole key example 'leisure' 
# #     get every type of leisure instalation, as a list returns several keys
    
#     string_keys = ''
#     for k in keys: 
#         string_keys += f"""node[{k}](area);"""
#     return string_keys

# def param_key_value(key, value):
    
#     #params defining the key and value, example 'leisure' 'parks' will only get parks
    
#     string_key = f"""\
#     node['{key}'='{value}'](area);
#     way['{key}'='{value}'](area);
#     rel['{key}'='{value}'](area);
#     """
#     return string_key

## imports

In [1]:
import requests
import json

In [2]:
#style working properly
# osm_keys_ok = """node["amenity"="pub"](area.berlin);
#             node["amenity"="bar"](area.berlin); 
#             """

## query function

In [34]:
overpass_url = "http://overpass-api.de/api/interpreter"

study_area = 'area["ISO3166-2"="DE-BE"]->.berlin'

keys_values_osm = {'amenity':['bbq','cafe']}


def param_nodes(keys):
    '''converts the dict into a string, returns a str'''
    osm_keys = ''
    for k,val in keys.items():
        for v in val:
            osm_keys += f"""node['{k}'='{v}'](area.berlin);"""
    return osm_keys
    
def param_ways(keys):
    '''converts the dict into a string, returns a str'''
    osm_keys = ''
    for k,val in keys.items():
        for v in val:
            osm_keys += f"""way['{k}'='{v}'](area.berlin);"""
    return osm_keys

def param_areas(keys):
    '''converts the dict into a string, returns a str'''
    osm_keys = ''
    for k,val in keys.items():
        for v in val:
            osm_keys += f"""area['{k}'='{v}'](area.berlin);"""
    return osm_keys


def query_params_osm(keys, feat):
    '''Adding keys and values as a dictionary, example: keys_values_osm = {'amenity':['bbq','cafe']}
    several values can added to a same key as a list, returns a dict
    arg feat = nodes, ways or areas'''
    
    if feat == 'ways':
        params = param_ways(keys)
        out_type = 'geom'
    if feat == 'areas':
        params = param_areas(keys)
        out_type = 'geom'
    if feat == 'nodes':
        params = param_nodes(keys)
        out_type = 'center'
        
    overpass_query = f"""
                    [out:json];
                    {study_area};
                    ({params}
                    );
                    out {out_type};
                    """
    response = requests.get(overpass_url, 
                            params={'data': overpass_query})
    return response.json()

In [35]:
keys_values_osm

{'amenity': ['bbq', 'cafe']}

In [21]:
param_nodes(keys_values_osm)

"node['amenity'='bbq'](area.berlin);node['amenity'='cafe'](area.berlin);"

In [22]:
data = query_params_osm(keys_values_osm, feat='nodes')

In [23]:
data

'\n                    [out:json];\n                    area["ISO3166-2"="DE-BE"]->.berlin;\n                    (node[\'amenity\'=\'bbq\'](area.berlin);node[\'amenity\'=\'cafe\'](area.berlin);\n                    );\n                    out center;\n                    '

In [24]:
len(data['elements'])

TypeError: string indices must be integers

# Params for Mobility

## Get dict for public transport

nodes

In [ ]:
public_transport = {'amenity':['bus_station'],
                    'bus_bay':'',
                   'highway':['bus_stop','platform'],
                   'public_transport':['stop_position','platform','station','stop_area'],
                   'railway':['station','tram_stop', 'subway_entrance']}

bike_infraestructure =  {'amenity':['bicycle_parking', 'bicycle_repair_station', 'bicycle_rental']}

In [ ]:
param_nodes(public_transport)

In [ ]:
query_publictransport = query_params_osm(public_transport, 'nodes')
query_bikeinfraestructure = query_params_osm(bike_infraestructure, 'nodes')

## Get dict for Cyclelanes and pedestrians

Ways queries

In [36]:
cycle_paths = {'bicycle':['designated'],
              }
# 'highway':['cycleway']

In [37]:
param_ways(cycle_paths)

"way['bicycle'='designated'](area.berlin);"

In [38]:
data_cycle = query_params_osm(cycle_paths, feat='ways')

In [39]:
data_cycle

{'version': 0.6,
 'generator': 'Overpass API 0.7.57 93a4d346',
 'osm3s': {'timestamp_osm_base': '2022-05-31T16:05:24Z',
  'timestamp_areas_base': '2022-05-31T11:06:54Z',
  'copyright': 'The data included in this document is from www.openstreetmap.org. The data is made available under ODbL.'},
 'elements': [{'type': 'way',
   'id': 4389576,
   'bounds': {'minlat': 52.4972925,
    'minlon': 13.2820626,
    'maxlat': 52.4977853,
    'maxlon': 13.2832468},
   'nodes': [26807995, 26807996],
   'geometry': [{'lat': 52.4972925, 'lon': 13.2820626},
    {'lat': 52.4977853, 'lon': 13.2832468}],
   'tags': {'bicycle': 'designated',
    'bridge': 'yes',
    'foot': 'designated',
    'highway': 'path',
    'layer': '1',
    'lit': 'no',
    'name': 'Trabener Steg',
    'name:etymology:wikidata': 'Q540919',
    'name:etymology:wikipedia': 'de:Traben-Trarbach',
    'segregated': 'no',
    'smoothness': 'good',
    'surface': 'concrete'}},
  {'type': 'way',
   'id': 4395584,
   'bounds': {'minlat': 52

# Params for Social Life

nodes

In [ ]:
eating = {'amenity':['cafe','restaurant', 'food_court', 'ice_cream']}

night_life = { 'amenity':['bar','pub','biergarten', 'nightclub', 'swingerclub', 'casino']}

culture = {'amenity':['social_centre','theatre','public_bookcase',
                      'fountain', 'events_venue', 'community_centre', 'cinema', 
                      'arts_centre', 'conference_centre']}

community = {'office': ['association','charity', 'coworking', 
                        'educational_institution', 'employment_agency', 'foundation', 
                        'ngo', ; 'political_party', 'research']}

# Params for Activities


nodes

In [ ]:
health_care = {'amenity':['baby_hatch','clinic','dentist',
                      'doctors', 'hospital', 'nursing_home', 'pharmacy', 
                      'social_facility', 'veterinary']}

public_service = {'amenity':['courthouse','fire_station','police',
                      'post_box', 'post_office', 'townhall']}

education = {'amenity':['college','kindergarten','language_school',
                      'library', 'music_school', 'school', 'university']}

economic = {'amenity':['atm', 'bank', 'bureau_de_change']}

#  Params for Comfort

## Node Queries

In [ ]:
comfort_spots = {'amenity':['bbq','bench','dog_toilet',
                      '	drinking_water', 'give_box', 'shelter', 'shower', 
                      'toilets', 'water_point', 'watering_place']}

leisure_spots = {'leisure':['bandstand','bird_hide','dog_park',
                      'firepit', 'swimming_pool', 'stadium', 'sports_centre', 
                      'pitch', 'picnic_table', 'fitness_centre'],
                 'historic':''}

trees = []


## Area querys

In [ ]:
leirsure_areas = {'leisure':['park','garden','swimming_area',
                      'playground', 'nature_reserve', 'marina'],
                 'landuse':['forest']}